# Building a Trustworthy LLM Summarization Bot with Galileo Observe

In this tutorial, we'll build a LLM based summarization bot and monitor the results in Galileo Observe.

This notebook pulls data from huggingface for its datasource and uses Open AI for LLM. Feel Free to change these sources as you'd like

## 1. Set-up of the environment

Let's start by installing the required libraries.

In [ ]:
! pip install galileo-observe openai ipywidgets

## 2. Set-up Galileo Clients

Next we will setup Galileo Observe client. You will need to enter 2 things - 
 - GALILEO API KEY: This is the API key used to connect to the client. You can fetch this from the console
 - OPENAI API KEY: For this notebook we are using Open AI so enter your Open AI Key here. If you are using some other model, you can skip this
 - Project Name - Define a name for the project

In [ ]:
import os
from galileo_observe import ObserveWorkflows

os.environ["GALILEO_CONSOLE_URL"] = "https://console.dev.rungalileo.io"
os.environ["GALILEO_API_KEY"] = "" # Enter Galileo Key here
os.environ["OPENAI_API_KEY"] = "" # Enter Open AI Key here
observe_logger = ObserveWorkflows(project_name="cookbook_summarization_monitoring")

## 3. Loading and Preparing Data

For this lab we will use a fictuous use case where we want to build a bot to summarize patient notes.

Now in order to build the bot, we will 
 - Fetch some patient notes from a website, 
 - Use an openAI powered LLM to summarize the notes
 - Monitor the LLM's responses with the help of Galileo Observe

In our case let's start by downloading some patient notes from a website.

In [3]:
from datasets import load_dataset

dataset = load_dataset("ncbi/Open-Patients")
df = dataset['train'].to_pandas()

Lets have a look at the data

In [ ]:
avg_doc_length = int(df['description'].apply(len).mean())
print(f'Average length of the documents is {avg_doc_length} characters.')
std_doc_length = int(df['description'].apply(len).std())
print(f'Standard deviation of the documents is {std_doc_length} characters.')

## 4. Run Inference with Galileo Observe

In [5]:
# Create a function to generate a response using Open AI

from openai import OpenAI
client = OpenAI()

def generate_response(prompt: str, history: list = [], model_name: str = "gpt-4o-mini"):
    
    response = client.chat.completions.create(
        model=model_name,
        messages=history + [{"role": "user", "content": prompt}],
        max_tokens=512,
        temperature=1,
        top_p=1
    )
    
    response_text = response.choices[0].message.content
    input_tokens = response.usage.prompt_tokens
    output_tokens = response.usage.completion_tokens
    total_tokens = response.usage.total_tokens
    
    return response_text, input_tokens, output_tokens, total_tokens

Here we define the model we want to use, and the system prompt for the LLM

In [7]:
def create_summarization_prompt(text: str) -> str:
    prompt = f"""Please provide a clear and concise summary of the following patient notes. 
Focus on key medical information, diagnoses, treatments, and important observations.
Keep the summary professional and maintain medical accuracy.

Patient Notes:
{text}"""
    return prompt


In [6]:
MODEL_ID = "gpt-4o-mini"

Now let's run the actual inference and log the information to Galileo! If you want to run the LLM summarization for more notes, set the sample size accordingly

In [8]:
patient_notes = df.sample(10)['description'].tolist()

In [ ]:
from tqdm import tqdm

for patient_note in tqdm(patient_notes):
    prompt = create_summarization_prompt(patient_note)

    # Create your workflow to log to Galileo.
    wf = observe_logger.add_workflow(input={"prompt": prompt}, name="Summarization", metadata={"env": "demo"})
    
    model_response, input_tokens, output_tokens, total_tokens = generate_response(prompt, model_name=MODEL_ID)

    # Log your llm call step to Galileo.
    wf.add_llm(
        input=prompt,
        output=model_response,
        model=MODEL_ID,
        input_tokens=input_tokens,
        output_tokens=output_tokens,
        total_tokens=total_tokens,
        metadata={"env": "demo"},
        name="Summarization",
    )

    # Conclude the workflow.
    wf.conclude(output={"output": model_response})

As a last step, we shall upload all the gathered information to Galileo

In [10]:
# Log the workflow to Galileo.
logged_workflows = observe_logger.upload_workflows()

You can have a look at the final results in the console via the link generated from the project

## Conclusion

Throughout this notebook, we have explored the process of creating and monitoring a summarization bot for a patient notes using GPT 4o mini via Open AI. We covered essential steps, including setting up the environment, loading and preparing patient notes, generating summaries, and logging to Galileo.
